# Seq2Seq 모델 구현 및 챗봇데이터 학습


In [ ]:
#Chatbot-torch clone
!git clone https://github.com/Chat-with-U/chatbot-pytorch.git

In [2]:
#konlpy 설치
!pip install konlpy

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [2]:
# mecab 설치 프로그램 clone
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [3]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [ ]:
# mecab 설치 프로그램 실행
!bash install_mecab-ko_on_colab_light_210108.sh

In [1]:
import os, sys
sys.path.append(os.pardir)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import optim
import re
import core.config as conf
from konlpy.tag import Mecab  # tweepy오류로 konlpy 직접 설치 필요, mecab별도 설치 readme 참고

가상 환경 생성<br> 
`conda create -n 가상환경이름 python=3.7`

PyTorch(Windows, Conda, CUDA 10.2)<br>
`conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch`

In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# 데이터 전처리를 위한 설정
Seq2Seq에서의 임베딩은 아래와 같이 추가 토큰을 사용하여 동작을 제어한다.

- `<PAD>`: 0, Padding, 짧은 문장을 채울 때 사용하는 토큰
- `<SOS>`: 1, Start of Sentence, 문장의 시작을 나타내는 토큰
- `<EOS>`: 2, End of Sentence, 문장의 끝을 나타내는 토큰
- `<UNK>`: 3, Unkown Words, 없는 단어를 나타내는 토큰

디코더 입력에 <SOS>가 들어가면 디코딩(문장)의 시작을 의미하고 출력에 <EOS>가 나오면 디코딩(문장)을 종료한다.
<br>

# 데이터 불러오기

In [9]:
path = conf.data_path
chatbot_data = pd.read_csv(f'{path}'+'ChatbotData.csv')
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
print('총 대화 쌍: ', len(chatbot_data))

총 대화 쌍:  11823


두 사람의 대화를 가정할 때 대화를 시작하는 대화를 question, 대답을 answer라 명명하겠습니다

이 챗봇의 경우 모든 대화를 1턴으로 가정하기 때문에 [대화 시작, 대답]을 대화의 끝으로 봅니다

In [12]:
question = chatbot_data.Q # Seq2Seq에 encoder_input
answer = chatbot_data.A # Seq2Seq에 decoder_input
total_utterances = pd.concat((question,answer)) #vocab을 만들기위한 전체 발화문장

# 형태소분석

챗봇 데이터 문장을 먼저 최소단위(우리말 형태소)로 tokenizing 해야합니다. 한국어는 KoNLPy라는 패키로 진행을 합니다.

(해당 예시에서는 mecab 사용)
(mecab외에도 Okt, komoran, kkma등 다른 형태소 분석기도 존재합니다. 각각 사용해보시고 차이점을 보는것도 재밌는 요소중 하나입니다)


*최소단위(형태소)로 tokenizing한다는 이야기는 <span style="color:red">형태소 단위로 문장을 쪼개는 것</span>을 의미합니다

In [13]:
pos_tagger = Mecab() #konlpy의 대표적인 형태소 분석기 mecab

단어 임베딩 vocab을 만들기위해 아래와같이 단어별로 나눈 뒤 <span style="color:red">vocab 리스트</span>에 넣어줍니다.

vocab 리스트는 갖고있는 모든 데이터셋의 단어 집합이라고 생각하시면 됩니다.

예를들어 갖고있는 문장이 

<span style="color:blue">'내가 그린 기린 그림은 긴 기린 그림이고 니가 그린 기린 그림은 짧은 기린 그림이다'</span> 한문장이라고 생각한다면

vocab list는 [내가, 그린, 기린, 그림, 은 ,긴, 이고, 니가, 짧은, 이다] 가 될 수 있습니다

vocab에는 중복되는 단어를 최소화 해야하며 vocab은 추후 정수 인코딩 부분에도 등장하기 때문에 개념을 잘 기억하고 넘어가는게 중요합니다

In [17]:
vocab = []

special_tokens = ['[PAD]', '[MASK]', '[START]', '[END]', '[UNK]']
for special_token in special_tokens:
    vocab.append(special_token)

for utterance in total_utterances:
    for eojeols in pos_tagger.pos(utterance,flatten=False, join=True): # 대화를 형태소 분석기로 나눈 뒤 어절단위로 쪼갠 후
        count = 0
        for token in eojeols:
            if count > 0:
                if token in vocab:
                    continue
                vocab.append('##' + token) # 어절의 뒤에 나오는 형태소에 ##을 붙여 앞의 형태소와 이어지는 형태소임을 명시합니다. ex) 학교에 -> [학교, "##에"]
            else:
                if token in vocab:
                    continue
                vocab.append(token)
                count += 1
        
vocab_size = len(vocab)

In [15]:
vocab_size

8665

vocab_size는 만든 vocab이 몇개의 단어 혹은 형태소를 갖고있는가 나타내는 지표입니다.

전체 11823개의 대화쌍의 데이터로 각 대화마다 2개의 문장이 존재한다고 하면 대략 2만4천개의 문장이 있다고 생각할 수 있습니다.

즉 8665개의 단어/형태소를 이용하여 2만4천개의 문장을 표현할 수 있다는 것을 의미합니다

# 토큰(단어/형태소) 인덱싱

지금까지 형태소 분석기를 이용하여 우리가 갖고있는 데이터를 몇개의 형태소로 표현할 수 있는지 알아보았습니다.

그렇다면 글자를 컴퓨터가 이해할 수 있는 숫자로 바꿔주는 작업이 필요합니다.

그 첫번째 단계는 정수 인코딩 입니다.

정수 인코딩을 그림으로 표현하면 다음과 같습니다.

<p align="center"><img src="https://github.com/Chat-with-U/chatbot-study/blob/main/img/int_encoding.png?raw=true"></p>

vocab에는 단어에 맞는 index값이 있고 각 문장별로 단어를 vocab의 index로 매칭하는 과정을 거치게 됩니다.

이는 추후 나오는 벡터화(vectorization)에서 다시한번 나오게 됩니다!

In [18]:
token2index = {token : index for index, token in enumerate(vocab)} # token(형태소)을 key, index를 value로 두어 특정 형태소를 index로 변환하는 테이블을 만듭니다.
index2token = {index : token for index, token in enumerate(vocab)} # 추후 모델을 통해 추론된 정수 index를 token(형태소)으로 변환하는 테이블을 만듭니다.

편의를 위해 WordHandler라는 클래스를 만들었습니다.

클래스의 encode 부분은 문장을 vocab의 index로 변환하는 함수, decode는 인덱스를 문장으로 변환하는 과정을 수행하는 함수입니다.

*decode without tag는 token뒤에 붙는 형태소 tag을 제거한 뒤 decode를 진행하는 함수입니다.

하지만 max_seq_len의 역할은 무엇일까요?

## max sequence length(최대 문장 길이)

cats & dogs, mnist 모델을 구현해보시거나 다른 이미지 분류 테스크를 진행해보신 분들이라면 image resize의 측면으로 접근이 가능합니다.

image분류 task에서 resize를 왜 진행하게 될까요? 

image의 사이즈가 제각기 다르다면 마지막으로 얻어낸 feature vector의 크기가 달라지게 되고 last feature map을 flatten했을 때 제각기 다른 parameter size를 갖게 될것입니다.

때문에 image의 사이즈를 통일하여 마지막 feature map의 사이즈를 통일시켜주게 됩니다.

NLP에선 max_seq_len가 이와 같은 역할을 하게됩니다.

문장의 길이가 전부 제각각이고 짧은 문장, 긴 문장을 하나의 문장 크기로 조정해주는 역할을 하게됩니다.

긴 문장은 max_seq_len보다 길다면 조금 자르고 짧은 문장은 앞서 정의한 '[PAD]' 토큰으로 채워주어 각 문장의 길이를 맞추어 모델에 동일한 input을 넣을 수 있도록 조정하는 역할을 수행합니다.

In [21]:
class WordHandler:
    def __init__(self, vocab, pos_tagger, token2index, index2token):
        self.vocab = vocab
        self.pos_tagger = pos_tagger
        self.token2index = token2index
        self.index2token = index2token
        
    def encode(self, sentence):
        encoded_vector = [self.token2index[token] if token in self.token2index else self.token2index['[UNK]']
                          for token in self.pos_tagger.pos(sentence, join= True)]
        
        return encoded_vector
    
    def decode(self, indice, join=True):
        decoded_vector = [self.index2token[index] for index in indice]
        
        return decoded_vector
    
    def decode_without_tag(self, indice):
        decoded_vector = ' '.join([self.index2token[index].split('/')[0] for index in indice])
        
        return decoded_vector
    
    @staticmethod
    def return_max_seq_len(sentences):
        max_seq_len = 0
        for sentence in sentences:
            max_seq_len = max(len(sentence), max_seq_len)
        
        return max_seq_len 
    
    # 이 handler는 주어진 데이터셋에서 가장 긴 문장길이를 max_seq_len로 return합니다.
    # 따라서 긴 길이의 문장이 잘려 손실이 발생하진 않지만 짧은 문장은 거의 [PAD]토큰으로 채워질 수 있습니다.
            
        
        

# Dataset

Seq2Seq 모델로 데이터를 전달하기 위한 Dataset을 구현하였습니다.

문장의 첫번째에 [START]토큰을 넣고 끝에 [END]토큰을 넣어 문장의 시작과 끝을 모델에 알려줍니다.

In [22]:
class ChitChatDataset(Dataset):
    def __init__(self, input_ids, output_ids, index2token, token2index, max_seq_len):
        self.input_ids = input_ids
        self.output_ids = output_ids
        self.index2token = index2token
        self.token2index = token2index
        self.max_seq_len = max_seq_len

    def __getitem__(self, idx):
        
        if len(self.input_ids[idx]) + 2 < self.max_seq_len:
            padding_block = self.max_seq_len - len(self.input_ids[idx]) + 2
            input = torch.LongTensor([self.token2index['[START]']] + 
                                     self.input_ids[idx] + 
                                     [self.token2index['[END]']] + 
                                     [self.token2index['[PAD]']] * padding_block)
        else:
            input = torch.LongTensor([self.token2index['[START]']] + 
                                     self.input_ids[idx] + 
                                     [self.token2index['[END]']])
        
        if len(self.output_ids[idx]) + 2 < self.max_seq_len:
            padding_block = self.max_seq_len - len(self.output_ids[idx]) + 2
            output = torch.LongTensor([self.token2index['[START]']] + 
                                     self.output_ids[idx] + 
                                     [self.token2index['[END]']] + 
                                     [self.token2index['[PAD]']] * padding_block)
        else:
            output = torch.LongTensor([self.token2index['[START]']] + 
                                     self.output_ids[idx] + 
                                     [self.token2index['[END]']])

        return input, output

    def __len__(self):
        return len(self.input_ids)

In [26]:
handler = WordHandler(vocab, pos_tagger, token2index, index2token)

모델에 들어갈 input과 output을 정의합니다.

input은 대화의 시작, output은 그에대한 대답으로 이루어집니다.

따라서 우리가 대화를 시작하면 챗봇은 그에 대한 대답을 하도록 학습되어지게 만들어집니다.

In [35]:
input_ids = question.map(handler.encode)
output_ids = answer.map(handler.encode)

In [37]:
print('사용자 대화: ', handler.decode_without_tag(input_ids[10]))
print('챗봇의 대답: ', handler.decode_without_tag(output_ids[10]))

사용자 대화:  SNS 보 면 나 만 빼 고 다 행복 해 보여
챗봇의 대답:  자랑 하 는 자리 니까요 .


In [38]:
max_seq_len = max(handler.return_max_seq_len(output_ids), handler.return_max_seq_len(input_ids))
max_seq_len

40

max_seq_len가 40이 나왔지만 조금 더 길게 대답하도록 만들어볼까요?

max_seq_len를 60으로 설정해보겠습니다.

물론 max_seq_len를 길게 설정한다고 모든 모델이 긴 대답을 하는 것은 아닙니다. 오히려 [PAD]토큰을 보는 빈도가 늘고 챗봇의 output이 [PAD]로 채우기만해도 loss가 떨어지는 현상이 발생할 가능성이 있습니다.

하지만 길게 설정되면 어떤 결과를 가져올까요? 여러분이 직접 조절해가며 결과와 loss값을 확인하는것도 재밌는 요소중 하나가 될것입니다.

In [39]:
# Dataset정의
chitchat_data = ChitChatDataset(input_ids, output_ids, index2token, token2index, 60)

In [40]:
# Dataloader 정의
chichat_dataloader = DataLoader(chitchat_data, batch_size= 100, shuffle = True)

아래의 코드는 Seq2Seq모델을 정의하는 부분입니다.

자세한 설명은 [링크]를 참조하세요!

In [43]:
class LSTMEncoder(nn.Module):
    def __init__(self, embedding_dim):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers=1 ,batch_first=True)

    def forward(self, encoder_embeds):
        hidden_and_cell = self.lstm(encoder_embeds)[1]
        return hidden_and_cell

    
class LSTMDecoder(nn.Module):
    def __init__(self, embedding_dim):
        super(LSTMDecoder, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, embedding_dim, num_layers=1 ,batch_first=True)

    def forward(self, decoder_embeds, hidden_and_cell):
        decoder_output = self.lstm(decoder_embeds, hidden_and_cell)[0]
        return decoder_output


class Seq2Seq(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Seq2Seq, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = LSTMEncoder(embedding_dim)
        self.decoder = LSTMDecoder(embedding_dim)
        self.vocab_proj = nn.Linear(embedding_dim, vocab_size)

    def forward(self, encode_input, decode_input):
        encoder_embeds = self.word_embedding(encode_input)
        decoder_embeds = self.word_embedding(decode_input)
        
        hidden_and_cell = self.encoder(encoder_embeds)
        decoder_output = self.decoder(decoder_embeds, hidden_and_cell)
        projected_output = self.vocab_proj(decoder_output)
        
        return projected_output


자 이제 학습을 위한 모든 준비가 끝났습니다. 과연 여러분의 챗봇은 어떤 대답을 하게될까요?

직접 확인해보시죠!

In [ ]:
model = Seq2Seq(vocab_size, 512).cuda()

criterion = nn.CrossEntropyLoss()
lr = 0.0001
optimizer = Adam(model.parameters(), lr)
step = 0



for epoch in tqdm(range(100)):
    for encode_input, decode_input, target in tqdm(chitchat_dataloader):
        encode_input = encode_input.cuda()
        decode_input = decode_input.cuda()
        target = target.cuda()
        step += 1
        optimizer.zero_grad()
        output = model(encode_input, decode_input)
        
        loss = 0
        for i in range(output.size()[0]):
            loss += criterion(output[i][:len(target[i])], target[i])
            
        loss.backward()
        optimizer.step()
        

    sample_input = handler.decode_without_tag(encode_input[0].tolist())
    sample_output = handler.decode_without_tag(output[0].argmax(-1).tolist())
    print(f'loss: {loss}')
    print(f'input_sentence: {sample_input}')
    print(f'output_sentence: {sample_output}')